# Case Study: Concrete Compressive Strength Prediction

In this case study, you'll solve a machine learning problem from start to finish.

The previous notebook, "analyze toydata", deals with a similar problem and can serve as a guideline for this exercise. You may also want to have a look at the [cheat sheet](https://franziskahorn.de/mlws_resources/cheatsheet.pdf) for more ideas and a concise overview of the relevant steps when developing a machine learning solution in any data science project. 

Feel free to get creative! 

In [ ]:
# first load some libraries that are needed later
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from scipy.optimize import minimize
# machine learning stuff
from sklearn.metrics import mean_absolute_error
from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import MaxAbsScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import ElasticNetCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn import tree
from sklearn.inspection import plot_partial_dependence, permutation_importance
from sklearn.manifold import TSNE
# interactive plotting (parallel coordinate plot)
import plotly.express as px
# suppress unnecessary warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# these "magic commands" are helpful if you plan to import functions from another script
# where you keep changing things, i.e., if you change a function in the script
# it will automagically be reloaded in the notebook so you work with the latest version
%load_ext autoreload
%autoreload 2

## Step 0: Load the data

The original data can be obtained from the [UCI ML data repository](https://archive.ics.uci.edu/ml/datasets/Concrete+Compressive+Strength). (If you're having trouble loading the xls file, you can also open it in Excel and then save it as a CSV file and use `pd.read_csv` instead.)

One data point in the dataset corresponds to one concrete mixture with the following variables:
- Mixture components (unit: kg in a m3 mixture):
    - Cement
    - Blast Furnace Slag
    - Fly Ash
    - Water
    - Superplasticizer
    - Coarse Aggregate
    - Fine Aggregate
- Age (number of days the concrete mixture hardened before the compressive strength was measured)
- Concrete compressive strength (unit: MPa; the main quality indicator for concrete (i.e., how strong it is))

Our goal is to **predict the compressive strength** of a concrete mixture based on the amounts of the different components. We will later filter the data for only the 28-day measurements, since this is the most important measurement to determine whether the concrete is within the norm constraints and okay to be used for construction.

In [ ]:
# load the file into a dataframe with pandas
df = pd.read_excel("../data/Concrete_Data.xls")
# look at the raw data (first 5 rows)
df.head()

In [ ]:
# the column names are not super convenient for an analysis, so let's rename them
RENAME_DICT = {
    'Cement (component 1)(kg in a m^3 mixture)': "cement",
    'Blast Furnace Slag (component 2)(kg in a m^3 mixture)': "slag",
    'Fly Ash (component 3)(kg in a m^3 mixture)': "fly_ash",
    'Water  (component 4)(kg in a m^3 mixture)': "water",
    'Superplasticizer (component 5)(kg in a m^3 mixture)': "plasticizer",
    'Coarse Aggregate  (component 6)(kg in a m^3 mixture)': "coarse_aggregate",
    'Fine Aggregate (component 7)(kg in a m^3 mixture)': "fine_aggregate", 
    'Age (day)': "age",
    'Concrete compressive strength(MPa, megapascals) ': "strength",
}
df = df.rename(columns=RENAME_DICT)
df.head()

## Step 1: Exploratory Analysis

Get a better understanding of the data and the problem:
- How are the individual variables distributed?
- Are any variables correlated? 
- Do you observe any patterns between the input and target variables? Do these make sense or is anything surprising?
- Anything else you should take into account when preprocessing the data later for the supervised learning part?

In [ ]:
# create some plots to better understand the data


## Step 2: Predict the 28-day Compressive Strength

Now that you've become more familiar with the dataset, it's time to tackle the real task, i.e., predict the 28-day compressive strength of a concrete mixture.

An evaluation pipeline is already set up below using a "stupid baseline" (= predicting the mean). Your task is to improve upon the performance by trying... 
- different models
- different preprocessing steps (e.g., transformations or feature engineering)
- hyperparameter tuning

Get creative :-)

**Tip:** Have a look at the [`make_pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html) function from sklearn to combine multiple steps (e.g., preprocessing and prediction model) into a single estimator object that can be applied to the original data.

In [ ]:
# the most important measurement to check whether the concrete adheres to
# the norm is after 28 days (-> almost half of our samples)
df["age"].value_counts()

In [ ]:
# for the prediction task we use only the 28-day samples 
df = df.loc[df["age"] == 28].reset_index()
df

In [ ]:
# define features (all input variables except age) and target
features = ['cement', 'slag', 'fly_ash', 'water', 'plasticizer', 'coarse_aggregate', 'fine_aggregate']
target = 'strength'

In [ ]:
# you might have noticed that there are a few sampes that have the exact 
# same feature values, but different strengths...
# we just assume that these were repeat measurements and take the average
df = df.groupby(features).mean().reset_index()
df

In [ ]:
# extract training and validation/test sets from this dataframe
# (normally, we would also set aside a final test set,
# but we only have very few samples here)
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=23)

In [ ]:
def eval_model(model, X_train, y_train, X_test, y_test):
    """
    Function to evaluate a trained regressor: prints R^2 and mean absolute error.
    
    Inputs:
        - model: the trained model
        - X_train, y_train: the training data
        - X_test, y_test: the test data
    """
    print(f"R^2 on training data: {model.score(X_train, y_train):.3f}")
    print(f"R^2 on test data: {model.score(X_test, y_test):.3f}")
    print(f"MAE on training data: {mean_absolute_error(y_train, model.predict(X_train)):.3f}")
    print(f"MAE on test data: {mean_absolute_error(y_test, model.predict(X_test)):.3f}")

In [ ]:
# train a dummy model
model = DummyRegressor()
model = model.fit(X_train, y_train)
# evaluate the model
eval_model(model, X_train, y_train, X_test, y_test)

In [ ]:
# now it's up to you: try an actual model and get better predictions!


## Step 3: Interpret the model

After you've found a model that achieves a decent performance, try to understand what it's doing.
- Calculate the permutation feature importance to see which features are most influential overall
- For the most important features, look at the partial dependence plot to see _how_ these features influence the outcome

Do these results make sense in terms of the actual physical process?

In [ ]:
# permutation feature importance


In [ ]:
# partial dependence plots


## Step 4: Optimization & What-If Analysis

When mixing concrete, water can usually be dosed rather flexibly.

Let's say our goal is to achieve a compressive strength after 28 days of 42.5 MPa. 

Use your prediction model on the test set to see whether the concrete is getting too strong or too weak and then change the water levels accordingly to make sure the production is on target.

You can run the code below as is, just make sure that `model` is an estimator object that also includes all necessary preprocessing steps (e.g., by using the `make_pipeline` function mentioned above).

Does your model help to get the production more on target?

In [ ]:
def optimize_water(model, x, value_min, value_max, target_strength=42.5):
    """
    Optimize the water content for a concrete mixture.
    
    Inputs:
        - model: the trained model
        - x: pandas dataframe row with one data point
        - value_min: minimum bound for water content
        - value_max: maximum bound for water content
        - target_strength: what we would like the output to be (default: 42.5)
    Returns:
        - water_org: original water content
        - water_new: optimized water content
        - pred_org: original strength prediction of the model
        - pred_new: strength prediction with optimized water content
    """
    # original situation
    water_org = x["water"].values[0]
    pred_org = model.predict(x)[0]
    print(f"original prediction with water content {water_org:.1f}: {pred_org:.2f} MPa")
    
    def _loss_fun(water_value):
        """
        Nested function (i.e., has access to all variables from the enclosing function)
        to compute the squared error between the models strength prediction with the given 
        water value and our target strength value.
        
        Inputs:
            - water_value: np.array with a single value, the proposed water content
        Returns:
            - loss: the squared error between the predicted and target strength
        """
        # insert the new value into our original data point
        new_x = x.copy()
        new_x["water"] = water_value[0]
        # predict strength with new water content
        pred_strength = model.predict(new_x)
        # optimization loss = squared difference to target value
        loss = (target_strength - pred_strength)**2
        return loss
    
    # use scipy's minimize function to find a value for 'water'
    # where the model predicts something close to our target value.
    # the start value for the optimization is the original water content.
    # to get realistic values, we additionaly specify bounds
    # based on the actual min/max values for the water content
    res = minimize(_loss_fun, np.array([water_org]), bounds=[(value_min, value_max)])
    # the optimized water content is stored in res.x (again a np.array)
    water_new = res.x[0]
    # check the final strength prediction
    new_x = x.copy()
    new_x["water"] = water_new
    pred_new = model.predict(new_x)[0]
    print(f"new prediction with water content {water_new:.1f}: {pred_new:.2f} MPa")
    return water_org, water_new, pred_org, pred_new
    

def optimize_water_all(model, X, target_strength=42.5):
    """
    Compute the optimized the water content for all data points.
    
    Inputs:
        - model: the trained model
        - X: pandas dataframe with input features for all data points
        - target_strength: what we would like the output to be (default: 42.5)
    Returns:
        - water_org_s: original water content for all data points
        - water_new_s: optimized water content for all data points
        - pred_org_s: original strength prediction of the model for all data points
        - pred_new_s: strength prediction with optimized water content for all data points
    """
    # bounds for optimization: known, realistic values for water content
    value_min, value_max = X["water"].min(), X["water"].max()
    # run the optimization for all data points
    water_org_s, water_new_s, pred_org_s, pred_new_s = [], [], [], []
    for i in range(len(X)):
        water_org, water_new, pred_org, pred_new = optimize_water(model, X.iloc[[i]], value_min, value_max, 
                                                                  target_strength)
        water_org_s.append(water_org)
        water_new_s.append(water_new)
        pred_org_s.append(pred_org)
        pred_new_s.append(pred_new)
    # convert lists to numpy arrays for easier plotting
    water_org_s, water_new_s = np.array(water_org_s), np.array(water_new_s)
    pred_org_s, pred_new_s = np.array(pred_org_s), np.array(pred_new_s)
    return water_org_s, water_new_s, pred_org_s, pred_new_s


def plot_optimization(water_org_s, water_new_s, pred_org_s, pred_new_s, y, target_strength=42.5):
    """
    Create two plots based on the results from optimize_water_all:
        1. What-if results after optimization: by changing the water content, the strength predictions
           should be closer to the target strength, even after correcting for prediction errors.
           The legend includes the MATD = mean absolute target deviation, i.e., how far away the
           respective points are from the target strength on average.
        2. Original and optimized water content and resulting strength increase/decrease.
    
    Inputs:
        - water_org_s: original water content for all data points
        - water_new_s: optimized water content for all data points
        - pred_org_s: original strength prediction of the model for all data points
        - pred_new_s: strength prediction with optimized water content for all data points
        - y: pandas dataframe with true compressive strength values for all data points
        - target_strength: what we would like the output to be (default: 42.5)
    """
    # convert y to a numpy array to make sure the indices match up with the other arrays
    target_org_s = y.to_numpy()
    
    # plot the optimization results
    plt.figure(figsize=(10, 5))
    # dashed line to indicated the target value we wanted
    plt.hlines(target_strength, 0, len(water_org_s), "k", "dashed", linewidth=1)
    # original target values as light blue dots
    plt.plot(target_org_s, "o", c="#53DDFE", alpha=0.8, 
             label=f"original y (MATD: {np.abs(target_org_s - target_strength).mean():.1f})")
    # predicted target values with original water content as blue x
    plt.plot(pred_org_s, "x", c="#007693", 
             label=f"predicted y (MATD: {np.abs(pred_org_s - target_strength).mean():.1f})")
    # predicted target values with optimized water content as orange x
    plt.plot(pred_new_s, "x", c="#A84801", 
             label=f"optimized predicted y (MATD: {np.abs(pred_new_s - target_strength).mean():.1f})")
    # since our original predictions are not perfect, we shift our optimized predictions
    # by the error we made on the original predictions -> plot as orange dots
    pred_new_corrected_s = pred_new_s + (target_org_s - pred_org_s)
    plt.plot(pred_new_corrected_s, "o", alpha=0.8, c="#FE9C53", 
             label=f"realistic optimized y (MATD: {np.abs(pred_new_corrected_s - target_strength).mean():.1f})")
    plt.xticks([], [])
    plt.xlabel("samples")
    plt.ylabel("compressive strength [MPa]")
    plt.title("What-If Analysis")
    plt.legend(loc=2, bbox_to_anchor=(1.02, 1), numpoints=1)
    
    # plot original and optimized water content
    plt.figure()
    # diagonal line -> original and optimized water content are the same
    plt.plot([water_new_s.min(), water_new_s.max()], [water_new_s.min(), water_new_s.max()], "k", alpha=0.5)
    # points above the line: more water than before
    # points below the line: less water than before
    # color of dot shows whether the optimization resulted in a reduction or increase in strength
    plt.scatter(water_org_s, water_new_s, c=pred_new_s-pred_org_s)
    plt.colorbar()
    plt.xlabel("original water content")
    plt.ylabel("optimized water content")
    plt.title("changes in water & strength");
    

In [ ]:
# run the optimization on the test set
water_org_s, water_new_s, pred_org_s, pred_new_s = optimize_water_all(model, X_test)

In [ ]:
# plot the results
plot_optimization(water_org_s, water_new_s, pred_org_s, pred_new_s, y_test)

## Step 5: Presentation of results
Clean up your code & think about which results you want to present and the story they tell:
- What have you learned about concrete production and how is this reflected in the data?
- What is the best model that you found & its performance?
- Which preprocessing steps had the most impact on the performance (for different models)?
- Which features were the most influential and how did they impact the model's prediction?
- What have you learned in this case study? Did any of the results surprise you?